# Notebook for the analysis of tHFA data

Changes mades:

- Version 1.2
    - Fixed the problem with the ISS for CHWs having all 100
- Version 1.1
    - Updated the outputs to include the underlying attributes
    - Used the QC excel files
    - Included a function to make the necessary pivot tables
- Version 1.0
    - Version used to start writing the report for tHFA in Malawi
- Version 0.9
    - Updated oxygen function to correct formulas based on the tHFA
    - Updated guidelines function to correct formula (there was a problem with using "value == target1 or target2")
    - Minor changes to facility score indicator
    - Changed high quality header name to "Quality_Score"
- Version 0.8
    - Included testing capacity indicator
- Version 0.7
    - Included all indicators except testing capacity
    - Sent to TGF for comparison


In [2]:
file_path = "./tHFA_MWI.xlsx"
pivot_out = "./malawi_pivot_v1.2.csv"
indicators_out = './tHFA-malawi-analysis_v1.2.csv'

In [3]:
import pandas as pd
import numpy as np

# Step 1: Load the CSV file into a pandas DataFrame

df = pd.read_excel(file_path, header=None)

# Step 2: Extract the first and second rows and save them in a dictionary
metadata = dict(zip(df.iloc[1], df.iloc[0]))

# Step 3: Set the second row as the header and reload the DataFrame // Changed with the new format
df.columns = df.iloc[0]
df = df[1:]
df.set_index('Q100', inplace=True)

# Step 4: Extract the first 26 columns into a separate DataFrame for survey details
survey_details_df = df.iloc[:, :11]
facility_details_df = pd.concat([df.iloc[:, 11:23], df.iloc[:, 29:37]], axis=1)
facility_details_df = facility_details_df.drop(columns=['FI_2','FI_3',"Q100a",'Q106_a','Q109-Latitude','Q109-Longitude','Q109-Altitude', 'Q109-Accuracy'])


test = df[:5]
test_fac_df = facility_details_df[:5]

In [187]:
# Function for the calculation of vacancy rate # Changed from the indicator definition in Annex 1

def calculate_vacancy(row):
    """RSSH/PP HRH-1: Vacancy rate"""
    funded_posts = float(row['tHFA_B046'])
    unfilled_posts = float(row['tHFA_B047'])
    if funded_posts != 999 and unfilled_posts != 999:
        score = ((unfilled_posts / funded_posts)) * 100 
    else: score = np.nan 
    
    return   pd.Series({
        'vacancy_score': score,
        'all_funded_posts': funded_posts,
        'all_unfilled_posts': unfilled_posts,
    })

In [188]:
# Function for calculation of oxygen therapy

def calculate_oxygen(row):
    """
    RSSH/PP RCS-1: Percentage of health facilities able to provide oxygen therapy related services
    
    Using only oxygen currently available in this unit and oxygen be brought to this unit from a different unit

    """
    provides_oxygen = 'YES' if row["R1323"] == 'YES' else "NO"
    obs_options = ['OBSERVED','REPORTED, NOT SEEN']
    
    if provides_oxygen == 'YES':
       avl = 100 if any ([row ['R1324'] == 'YES', row['R1326'] == 'YES']) and all([row[f'R1325_0{i}A'] in obs_options for i in ['4','5','6']]) and all([row[f'R1325_0{i}B'] == 'YES' for i in ['4','5','6']]) else 0
       avo = 100 if all([row[f'R1322_09A'] in obs_options , row[f'R1322_09B'] == 'YES' ]) else 0
    else:
        avl = np.nan
        avo = np.nan
    
    score = np.nanmean([avl,avo])

    return pd.Series({
        'oxygen_score': score,
        'oxygen_avl': avl,
        'oxygen_avo': avo,

    })

# see = test.apply(calculate_oxygen, axis=1)
# see


In [189]:
# Function for calculation of CHW paid on time and in full

def calculate_CHW_paid(row):
    
    """

    RSSH/PP HRH-3: Percentage of community health workers remunerated on time and in-full
    
    Paid is calculated with 'tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023' and all need to be present to have the value of 100

    """
    chw_count = sum(1 for col in ['CHW_begin_1', 'CHW_begin_2', 'CHW_begin_3', 'CHW_begin_4', 'CHW_begin_5'] if row[col] == 'YES')

    chw_paid_scores = []

    paid_columns = ['tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023']

    for i in range(1, chw_count + 1):
        chw_i_score = 0
        for col in paid_columns:
            interview_col = f'{col}_{i}'
            if row[interview_col] == 'YES': 
                chw_i_score = chw_i_score + 1
        chw_paid = 100 if chw_i_score == 4 else 0  
        chw_paid_scores.append(chw_paid)

    chw_paid_score = np.nanmean(chw_paid_scores)
    return pd.Series({
        # 'CHW_count': chw_count,
        'CHW_paid': chw_paid_score,
        'CHW_scores': chw_paid_scores  
        })

In [190]:
# Function for calculation of facility composite indicator 

def calculate_facility_composite(row):
    
    """

    RSSH/PP HRH-6: Percentage of facilities providing effective services

    Composite facility level indicator with seven components: 
    1)	% of facilities observed to provide integrated services at ANC (TB, malaria, HIV) at the time of visit; 
    2)	Provider availability (absence rate on day of visit);
    3)	Provider caseload (number of outpatient visits per clinician per day);
    4)	ANC dropout rate; 
    5)	DPT dropout rate; 
    6)	Treatment completion rate for new TB cases;
    7)	Twelve-month retention on ART


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

    outpatients_ystrdy = float(row['tHFA_B034'])
    clincians_present = float(row['tHFA_B026'])
    clinicians_today = float(row['tHFA_B025'])

    # Attribute 1 (Integrated HTM with ANC)
    if provides_anc > 0:
        htm_anc_fxn = calculate_HTM_ANC(row)
        htm_anc_score = htm_anc_fxn['HTM-ANC']
    else:
        htm_anc_score = np.nan


    # Attribute 2 (Clincian presence)
    if clinicians_today > 0 and clinicians_today != 999 and clincians_present != 999:
        presence = (clincians_present/clinicians_today)*100
    else: presence = np.nan 


    # Attribute 3 (Caseload)
    if clincians_present > 0 and outpatients_ystrdy != 999 and clincians_present != 999:
        caseload = (outpatients_ystrdy/clincians_present)*5
        if caseload >100:
            caseload = 100 
    else: caseload = np.nan

    

    # Attribute 4 (ANC dropout rate)
    anc_scores = []
    if provides_anc > 0:
        anc_count = sum(1 for col in ['RR_ANC_Begin_1', 'RR_ANC_Begin_2', 'RR_ANC_Begin_3', 'RR_ANC_Begin_4', 'RR_ANC_Begin_5'] if row[col] == 'YES')
        for i in range(1, anc_count + 1):            
            if int(row[f'Q13004_7_{i}']) == 99:
                anc_scores.append(np.nan)
            elif int(row[f'Q13004_7_{i}']) > 3 :
                anc_scores.append(100) 
            else: anc_scores.append(0)

        anc_score = np.nanmean(anc_scores)

    else:
        anc_score = np.nan


    # Attribute 5 (DTP dropout rate)
    DPT_dropout = np.nan
    if provides_immu > 0:
        if float(row['tHFA_B032']) != 999 and float(row['tHFA_B032']) != 0 and float(row['tHFA_B033']) != 999:
            DPT_dropout = (float(row['tHFA_B032']) - float(row['tHFA_B033']))/float(row['tHFA_B032'])
            DPT_score = (1-DPT_dropout) * 100
        else:
            DPT_score = np.nan
    
    else: 
        DPT_score = np.nan


    # Attribute 6 (treatment completion rate for new TB clients)
    if provides_TB > 0:
        tb_count = sum(1 for col in ['RR_TB_Begin_1', 'RR_TB_Begin_2', 'RR_TB_Begin_3', 'RR_TB_Begin_4', 'RR_TB_Begin_5'] if row[col] == 'YES')
        tb_trtmnt_scores = []
        
        for i in range(1, tb_count + 1):
            tb_trtmnt = np.nan
            if row[f'Q13506_04_{i}'] == 'YES, CLIENT WAS CURED OR COMPLETED TREATMENT':
                tb_trtmnt = 1
            elif row[f'Q13506_04_{i}'] == np.nan :
                tb_trtmnt = np.nan
            else: 
                tb_trtmnt = 0

            tb_trtmnt_scores.append(tb_trtmnt)
        
        tb_trtmnt_score = (np.nansum(tb_trtmnt_scores)/tb_count) * 100
    
    else: 
        tb_trtmnt_score = np.nan
        tb_trtmnt_scores = []

    # Attribute for ART 12 month retention
    if provides_ART > 0:   
        art_count = sum(1 for col in ['RR_ART_Begin_1', 'RR_ART_Begin_2', 'RR_ART_Begin_3', 'RR_ART_Begin_4', 'RR_ART_Begin_5'] if row[col] == 'YES')
        art_retnt_scores = []
        for i in range(1, art_count + 1):
            art_retnt = np.nan
            if row[f'tHFAG002_{i}'] == 'NO': 
                art_retnt = np.nan
            elif row[f'tHFAG002_{i}'] == 'YES' and row[f'tHFAG005_{i}'] == 'YES':
                art_retnt = 100
            else:
                art_retnt = 0
            
            art_retnt_scores.append(art_retnt)
        art_retnt_score = np.nanmean(art_retnt_scores)
    else:
        art_retnt_score = np.nan
        art_retnt_scores = []
    

    # Aggregation of scores
    attribute_scores = [htm_anc_score,presence,caseload,anc_score,DPT_score,tb_trtmnt_score,art_retnt_score]
    comp_score = np.nanmean(attribute_scores)
    
    
    return pd.Series({
        'facility_composite': comp_score,
        'HTM-ANC': htm_anc_score,
        'presence_score': presence,
        'caseload': caseload,
        'ANC_score': anc_score,
        'DPT_score': DPT_score,
        'TB_treatment_completion': tb_trtmnt_score,
        'ART_retention': art_retnt_score,
        
        })


# see = test.apply(calculate_facility_composite, axis=1)
# see

In [191]:
# Function for calculation of integrated HTM for ANC clients 

def calculate_HTM_ANC(row):
    
    """

    Pregnant women receive the following services in an integrated fashion: 
    HIV counseling and testing, access to ARVs for HIV positive women, IPTp, 
    and TB screening.


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

       
    htm_anc_scores = []
    hiv_testing_scores = []
    ART_access_scores = []
    iptp_access_scores = []
    TB_screening_scores = []
    if provides_anc > 0:
        anc_count = sum(1 for col in ['RR_ANC_Begin_1', 'RR_ANC_Begin_2', 'RR_ANC_Begin_3', 'RR_ANC_Begin_4', 'RR_ANC_Begin_5'] if row[col] == 'YES')
        for i in range(1, anc_count + 1):            
            client_anc_scores = []
            # Attribute 1: HIV testing and counselling
            if row[f'Q13009_1_{i}'] == 'YES':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F003_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F003_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)
            
            hiv_testing_scores.append(client_anc_scores[-1])

            # Attribute 2: Access to ART
            if row[f'tHFA_F001_{i}'] == 'NO':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)
            ART_access_scores.append(client_anc_scores[-1])

            # Attribute 3: Access to iptp
            if row[f'Q13007_3_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'Q13007_3_{i}'] == 'NO':
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)
            iptp_access_scores.append(client_anc_scores[-1])

            # Attribute 4: Access to TB screening
            tb_values = [row[f'TBSF002_A_{i}'],
                         row[f'TBSF002_B_{i}'], 
                         row[f'TBSF002_C_{i}'],
                         row[f'TBSF002_D_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_anc_scores.append(100)
            else:
                client_anc_scores.append(0)
            TB_screening_scores.append(client_anc_scores[-1])

            client_anc_score = np.nanmean(client_anc_scores)
            htm_anc_scores.append(client_anc_score)
        

        htm_anc_score = np.nanmean(htm_anc_scores)

    else:
        htm_anc_score = np.nan

    hiv_testing_score = np.nanmean(hiv_testing_scores) if len(hiv_testing_scores)>0 else np.nan 
    ART_access_score = np.nanmean(ART_access_scores) if len(ART_access_scores)>0 else np.nan 
    iptp_access_score = np.nanmean(iptp_access_scores) if len(iptp_access_scores)>0 else np.nan 
    TB_screening_score = np.nanmean(TB_screening_scores) if len(TB_screening_scores)>0 else np.nan 

    return pd.Series({
        'HTM-ANC': htm_anc_score,
        # 'htm-anc-check': htm_anc_scores,
        'hiv testing': hiv_testing_score,
        'ART access': ART_access_score,
        'iptp scores': iptp_access_score,
        'TB screening': TB_screening_score
        })

# see = test.apply(calculate_HTM_ANC, axis=1)
# see

In [192]:
# Function for evaluating testing capacity
def calculate_testing_capacity(row):
    
    """

   Description
   RSSH/PP LAB-5: “Percentage of health facilities that have an appropriate set of diagnostics 
   for their healthcare facility level, based on adapted WHO model list of essential in vitro diagnostics (EDL 3)”


    This needs data for the EDL tests from the 
   
    """

    # Responses for facilities with labs
    test_facility_questions = [f'EDL_RESPONSE_0{i}' for i in range(1,10)]
    test_facility_questions.extend([f'EDL_RESPONSE_{i}' for i in range(10,28)])
    
    # Responses for facilities without labs 
    all_facility_questions = test_facility_questions + ['EDL_RESPONSE_28']
    remove = [f'EDL_RESPONSE_{i}' for i in range(10,19)] + [f'EDL_RESPONSE_0{i}' for i in range(8,10)] + [f'EDL_RESPONSE_{i}' for i in range(22,28)]
    remove_set = set(remove)
    non_test_facility_questions = [thing for thing in all_facility_questions if thing not in remove_set]
    

    test_capac_scores = []
    lab_score = np.nan
    no_lab_score = np.nan

    if row['R2900'] == 'YES, WITH LABORATORY IN THE FACILITY':
        for question in test_facility_questions:
            if row[question] == 'YES': test_capac_scores.append(100)
            elif row[question] == 'NO': test_capac_scores.append(0)
            else: test_capac_scores.append(np.nan)
        
        lab_score = np.nanmean(test_capac_scores) 

    elif row['R2900'] == 'YES, WITHOUT LABORATORY IN THE FACILITY':
        for question in non_test_facility_questions:
            if row[question] == 'YES': test_capac_scores.append(100)
            elif row[question] == 'NO': test_capac_scores.append(0)
            else: test_capac_scores.append(np.nan)
        
        no_lab_score = np.nanmean(test_capac_scores) 
        
    elif row['R2900'] == 'NO LABORATORY TESTS PERFORMED':
        test_capac_scores.append(np.nan)
    
    
    test_capac_score = np.nanmean(test_capac_scores)





    return pd.Series({
        'test-capac': test_capac_score,
        'test-capac_check': test_capac_scores,
        'with lab': lab_score,
        'without lab': no_lab_score,
        })


# see = test.apply(calculate_testing_capacity, axis=1)
# see

In [193]:
# Function for guideline availability

def calculate_guideline_availability(row):
    
    """

   “Percentage of facilities with written and updated clinical guidelines for HIV, TB, malaria and/or 
   PHC (based on the services provided) developed by the national or sub-national government 
   (as appropriate by the country context)
   

    Returns a guideline availability score that is a np.nanmean of the scores for the relevant diseases 
    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

    guidelines = [provides_hiv_test, 
                  provides_ART,
                  provides_TB,
                  provides_malaria,
                  provides_anc,
                  provides_immu,
                  provides_imci]
    
    guideline_cols = ['R2308_01',
                      'R2315B_01',
                      'R2419_01',
                      'R1406',
                      'R1817_01',
                      'R2119_01',
                      'R2005_01']
    guideline_scores = []

    for i in range(len(guidelines)): 
        if guidelines[i] > 0 :
            if row[guideline_cols[i]] in ['YES, OBSERVED','YES, REPORTED, NOT SEEN']:
                guideline_scores.append(100)
            else: 
                guideline_scores.append(0)
        else: 
            guideline_scores.append(np.nan)
    
    
    guideline_score = np.nanmean(guideline_scores)

    return pd.Series({
        'guideline_score': guideline_score,
        # 'guideline_check': guideline_scores,
        'guide_HIV':100 if any([guideline_scores[0] == 100,guideline_scores[1] == 100]) else 0,
        'guide_TB': guideline_scores[2],
        'guide_malaria': guideline_scores[3],
        'guide_anc': guideline_scores[4],
        'guide_immu': guideline_scores[5],
        'guide_imci': guideline_scores[6],
         })

# see = test.apply(calculate_guideline_availability, axis=1)
# see


In [194]:
# Function for  checking community-led mechanism 
# # Unclear how don't knows should be treated here
def calculate_CLM(row):
    
    """

   Indicator for CSS-3: “Percentage of health service delivery 
   sites with a community-led monitoring mechanism in place”

   Returns:
    0 - 100 based on number of strongly agree or agree responses

    """

    questions = ['tHFA_B021', 'tHFA_B022', 'tHFA_B023', 'tHFA_B024', 'tHFA_B035', 'tHFA_B045']
    agree_count = sum(1 for q in questions if row[q] in ['STRONGLY AGREE', 'AGREE'])
    score = (agree_count / len(questions)) * 100


    return pd.Series({
        'CLM_score': score,      
        })

In [195]:
# Function to calculate system readiness

def calculate_system_CHW_old(row):
    
    """

   KPI S5: “Systems readiness for CHWs”


    """

    chw_count = sum(1 for col in ['CHW_begin_1', 'CHW_begin_2', 'CHW_begin_3', 'CHW_begin_4', 'CHW_begin_5'] if row[col] == 'YES')
    
    paid_columns = ['tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023']

    chw_scores = []

    chw_atttributes = {
        	'fac_supvsn_scores': [],
            'fac_contract_scores': [],
            'fac_paid_scores': [],
            'fac_jobtools_scores': [],
    }

    for i in range(1, chw_count+1):
        chw_i_score = 0
        chw_i_scores = []

        # Attribute 1 (Integrated Supportive Supervision)
        iss_score = 0
        a = 3
        chw_services =  [f'tHFA_E00{a}_A_{i}',f'tHFA_E00{a}_B_{i}', f'tHFA_E00{a}_C_{i}', f'tHFA_E00{a}_D_{i}',f'tHFA_E00{a}_E_{i}']
        for a in [3]:
            expected = sum(1 for disease in chw_services if row[disease] == 'YES')
        for a in [5,7,9]:
            sprvsn = sum(1 for service in chw_services if row[service] == 'YES' )
            if sprvsn == expected: iss_score = iss_score+1
        chw_iss = 100 if iss_score > 0 else 0
        chw_i_scores.append(chw_iss)
        chw_atttributes['fac_supvsn_scores'].append(chw_iss)

        # Attribute 2 (Presence of contract)
        contract = [f'tHFA_E0{b}_{i}' for b in range(10,20)]
        contract_scores = []
        for question in contract:
            if row[question]=='YES': contract_scores.append(100)
            elif row[question]=='NO': contract_scores.append(0)
            else: contract_scores.append(np.nan)
        chw_contract = 100 if np.nanmean(contract_scores) == 100 else 0
        chw_i_scores.append(chw_contract)
        chw_atttributes['fac_contract_scores'].append(chw_contract)
        
        # Attribute 3 (Payment accoording to contract)
        paid_score = 0 
        for col in paid_columns:
            interview_col = f'{col}_{i}'
            if row[interview_col] == 'YES': 
                paid_score = paid_score + 1
        chw_paid = 100 if paid_score == 4 else 0  
        chw_i_scores.append(chw_paid)
        chw_atttributes['fac_paid_scores'].append(chw_paid)

        # Attribute 4 (Job tools)
        tools = [f'tHFA_E0{b}_{i}' for b in [25,27,29]]
        tools_scores = []
        for question in tools:
            if row[question]=='YES': tools_scores.append(100)
            elif row[question]=='NO': tools_scores.append(0)
            else: tools_scores.append(np.nan)
        chw_tools = 100 if np.nanmean(tools_scores) == 100 else 0
        chw_i_scores.append(chw_tools)
        chw_atttributes['fac_jobtools_scores'].append(chw_tools)

        chw_i_score = np.nanmean(chw_i_scores)
        chw_scores.append(chw_i_score)

    chw_score = np.nanmean(chw_scores)
    
    chw_means = {}

    # Loop through the dictionary and calculate the mean for each score list
    for key, value in chw_atttributes.items():
        mean_key = key.replace('_scores', '_score')  # Create a new key for the mean
        chw_means[mean_key] = np.nanmean(value)


    return pd.Series({
        'system_CHW_score': chw_score,
        'chw_show': chw_scores,
        'chw_iss': chw_means['fac_supvsn_score'],
        'chw_contract': chw_means['fac_contract_score'],
        'chw_paid': chw_means['fac_paid_score'],
        'chw_tools': chw_means['fac_jobtools_score'],
        'chw_iss_deep': chw_atttributes['fac_supvsn_scores']

        })


see = test.apply(calculate_system_CHW_old, axis=1)
see

In [99]:
# Function to calculate system readiness

def calculate_system_CHW(row):
    
    """

   KPI S5: “Systems readiness for CHWs”


    """

    chw_count = sum(1 for col in ['CHW_begin_1', 'CHW_begin_2', 'CHW_begin_3', 'CHW_begin_4', 'CHW_begin_5'] if row[col] == 'YES')
    
    paid_columns = ['tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023']

    chw_scores = []

    chw_atttributes = {
        	'fac_supvsn_scores': [],
            'fac_contract_scores': [],
            'fac_paid_scores': [],
            'fac_jobtools_scores': [],
    }

    for i in range(1, chw_count+1):
        chw_i_score = 0
        chw_i_scores = []

        # Attribute 1 (Integrated Supportive Supervision)
        expected = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E003_{b}_{i}'] == 'YES']
        month_1 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E005_{b}_{i}'] == 'YES']
        month_2 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E007_{b}_{i}'] == 'YES']
        month_3 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E009_{b}_{i}'] == 'YES']

        chw_iss = 100 if any([set(globals()[f'month_{j}']) == set(expected) for j in [1,2,3]]) else 0
        chw_i_scores.append(chw_iss)
        chw_atttributes['fac_supvsn_scores'].append(chw_iss)

        # Attribute 2 (Presence of contract)
        contract = [f'tHFA_E0{b}_{i}' for b in range(10,20)]
        contract_scores = []
        for question in contract:
            if row[question]=='YES': contract_scores.append(100)
            elif row[question]=='NO': contract_scores.append(0)
            else: contract_scores.append(np.nan)
        chw_contract = 100 if np.nanmean(contract_scores) == 100 else 0
        chw_i_scores.append(chw_contract)
        chw_atttributes['fac_contract_scores'].append(chw_contract)
        
        # Attribute 3 (Payment accoording to contract)
        paid_score = 0 
        for col in paid_columns:
            interview_col = f'{col}_{i}'
            if row[interview_col] == 'YES': 
                paid_score = paid_score + 1
        chw_paid = 100 if paid_score == 4 else 0  
        chw_i_scores.append(chw_paid)
        chw_atttributes['fac_paid_scores'].append(chw_paid)

        # Attribute 4 (Job tools)
        tools = [f'tHFA_E0{b}_{i}' for b in [25,27,29]]
        tools_scores = []
        for question in tools:
            if row[question]=='YES': tools_scores.append(100)
            elif row[question]=='NO': tools_scores.append(0)
            else: tools_scores.append(np.nan)
        chw_tools = 100 if np.nanmean(tools_scores) == 100 else 0
        chw_i_scores.append(chw_tools)
        chw_atttributes['fac_jobtools_scores'].append(chw_tools)

        chw_i_score = np.nanmean(chw_i_scores)
        chw_scores.append(chw_i_score)

    chw_score = np.nanmean(chw_scores)
    
    chw_means = {}

    # Loop through the dictionary and calculate the mean for each score list
    for key, value in chw_atttributes.items():
        mean_key = key.replace('_scores', '_score')  # Create a new key for the mean
        chw_means[mean_key] = np.nanmean(value)


    return pd.Series({
        'system_CHW_score': chw_score,
        'chw_show': chw_scores,
        'chw_iss': chw_means['fac_supvsn_score'],
        'chw_contract': chw_means['fac_contract_score'],
        'chw_paid': chw_means['fac_paid_score'],
        'chw_tools': chw_means['fac_jobtools_score'],
        'chw_iss_deep': chw_atttributes['fac_supvsn_scores']

        })


see = test.apply(calculate_system_CHW, axis=1)
see

,system_CHW_score,chw_show,chw_iss,chw_contract,chw_paid,chw_tools,chw_iss_deep
Q100,,,,,,,
MWI_000193,31.25,"[25.0, 0.0, 75.0, 25.0]",50.0,25.0,25.0,25.0,"[0, 0, 100, 100]"
MWI_001282,85.00,"[75.0, 100.0, 75.0, 75.0, 100.0]",60.0,100.0,80.0,100.0,"[100, 100, 0, 0, 100]"
MWI_000166,75.00,"[100.0, 50.0, 75.0, 75.0, 75.0]",100.0,20.0,100.0,80.0,"[100, 100, 100, 100, 100]"
MWI_001312,90.00,"[100.0, 100.0, 100.0, 100.0, 50.0]",100.0,80.0,100.0,80.0,"[100, 100, 100, 100, 100]"
MWI_001109,65.00,"[75.0, 75.0, 75.0, 50.0, 50.0]",100.0,20.0,80.0,60.0,"[100, 100, 100, 100, 100]"


In [98]:
expected = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E003_{b}_{i}'] == 'YES']
month_1 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E005_{b}_{i}'] == 'YES']
month_2 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E007_{b}_{i}'] == 'YES']
month_3 = [f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E009_{b}_{i}'] == 'YES']

chw_iss = 100 if any([set(globals()[f'month_{j}']) == set(expected) for j in [1,2,3]]) else 0
print([set(globals()[f'month_{j}']) == set(expected) for j in [1,2,3]])
# set(month_1).issubset(set(expected))
# test.iloc[1][r]
print(set(expected), set(month_1), set(month_2), set(month_3))
print(chw_iss)

[True, False, True]
{'B', 'A', 'E', 'D', 'C'} {'B', 'A', 'E', 'D', 'C'} set() {'B', 'A', 'E', 'D', 'C'}
100


In [33]:
# Function for high quality service
def calculate_service_quality(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_ANC = 1 if row['R1810'] == 'YES' else 0



    # Calculate HW competence scores
    # ANC scores
    hw_scores = []
    anc_scores = []
    tb_scores = []
    art_scores = []

    if provides_ANC > 0:
        anc_count = sum (1 for col in [f'RR_ANC_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')
        
        for i in range(1, anc_count+1):
            client_anc_scores = []
             # Attribute 1: HIV testing and counselling
            if row[f'Q13009_1_{i}'] == 'YES':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F003_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F003_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)

            # Attribute 2: Access to ART
            if row[f'tHFA_F001_{i}'] == 'NO':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)

            # Attribute 3: Access to iptp
            if row[f'Q13007_3_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'Q13007_3_{i}'] == 'NO':
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)

            # Attribute 4: Access to TB screening
            tb_values = [row[f'TBSF002_A_{i}'],
                         row[f'TBSF002_B_{i}'], 
                         row[f'TBSF002_C_{i}'],
                         row[f'TBSF002_D_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_anc_scores.append(100)
            else:
                client_anc_scores.append(0)
            
            # Attritube 5, 7, 8, 9, 10, 11
            # Integrated ANC and SRH, Blood pressure, Folic acid, Danger signs
            # Intestinal worms, and hemoglobin
            q = ['13005_1',
                    '13004_8',
                    '13004_10',
                    '13004_11',
                    '13005_2',
                    '13005_5']
            
            for attr in q: 
                if row[f'Q{attr}_{i}'] == 'YES':
                    client_anc_scores.append(100)
                elif row[f'Q{attr}_{i}'] == 'NO':
                    client_anc_scores.append(0)
                else: 
                    client_anc_scores.append(np.nan)

            # Attribute 6: ANC age
            if int(row[f'Q13004_1_{i}']) >= 32:
                client_anc_scores.append(100)
            elif int(row[f'Q13004_1_{i}']) < 32:
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)
            
            client_anc_score = np.nanmean(client_anc_scores)
            anc_scores.append(client_anc_score)

    if provides_ART > 0:
        art_count = sum (1 for col in [f'RR_ART_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')
        
        for i in range(1, art_count+1):
            client_art_scores = []
            # Attribute 1: TB screening
            tb_values = [row[f'Q13408_01_{i}'],
                        row[f'Q13408_02_{i}'], 
                        row[f'Q13408_03_{i}'],
                        row[f'tHFAG003_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_art_scores.append(100)
            else:
                client_art_scores.append(0)
            
            #Attribute 2 : TB treatment for ART patients
            if row[f'Q13408_08_{i}'] == 'ACTIVE TB':
                client_art_scores.append(100) if row[f'Q13408_09_{i}'] == 'YES' else client_art_scores.append(0) 
            else: client_art_scores.append(np.nan)

            # Attribute 3: BP measured
            bp = 'tHFAG001'
            if row[f'{bp}_{i}'] == 'YES':
                client_art_scores.append(100)
            elif row[f'{bp}_{i}'] == 'NO':
                client_art_scores.append(0)
            else: 
                client_art_scores.append(np.nan)
            
            client_art_score = np.nanmean(client_art_scores)
            art_scores.append(client_art_score)
        
    if provides_TB >0 :
        tb_count = sum (1 for col in [f'RR_TB_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')

        for i in range(1,tb_count+1):
            client_tb_scores=[]

            # Attribute 1 and 2:
            # HIV test and symptpms
            tb = ['Q13507_01',
                'Q13510_01']
        
            for attr in tb: 
                if row[f'{attr}_{i}'] == 'YES':
                    client_tb_scores.append(100)
                elif row[f'{attr}_{i}'] == 'NO':
                    client_tb_scores.append(0)
                else: 
                    client_tb_scores.append(np.nan)
            
            client_tb_score = np.nanmean(client_tb_scores)
            tb_scores.append(client_tb_score)
    
    

    hw_scores = anc_scores + art_scores + tb_scores

    hw_score = np.nanmean(hw_scores) if len(hw_scores) > 0 else np.nan
   

    def calculate_column_score(value, column):
        if column in ['tHFA_D004', 'tHFA_D005']:
            return 100 if value == 'YES' else 0
        elif column == 'tHFA_D006':
            return 0 if value == 'YES' else 100
        elif column == 'Q13004_1':
            return 
        elif column in  ["tHFA_D011_TOO_EXPENSIVE", 
                        'tHFA_D011_TOO_FAR',
                        'tHFA_D011_LONG_WAITING_TIME',
                        'tHFA_D011_FACILITY_OFTEN_CLOSED',
                        'tHFA_D011_HEALTH_WORKERS_NOT_AVAILABLE',
                        'tHFA_D011_HEALTH_WORKERS_NOT_RESPECTFUL',
                        'tHFA_D011_LACK_OF_TRANSPORT_TO_REACH_FACILITY',
                        'tHFA_D011_POOR_QUALITY_OF_CARE']:
            return 0 if any(option in value for option in ['1']) else 100
        elif column in ['PREQ132', 'PREQ122', 'PREQ125', 'PREQ119', 'PREQ129']:
            scale = {'NEVER': 0, 'RARELY': 25, 'SOMETIMES': 50, 'OFTEN': 75, 'ALWAYS': 100}
            return scale.get(value, np.nan)
        else:
            return 100 if value == 'YES' else 0

    # Calculate Patient centeredness scores
    patient_centeredness_columns = [
        'PREQ132', 
        'PREQ122', 
        'PREQ125', 
        'PREQ119', 
        'PREQ129',
        'tHFA_D003', 
        'tHFA_D004', 
        'tHFA_D006', 
        'tHFA_D011'
    ]
    patient_count = sum(1 for col in [f'PE_Begin_{i}' for i in range(1,6)] if row[col] == 'YES')
    pc_scores = []
    
    for i in range(1, patient_count + 1):
        for col in patient_centeredness_columns:
            interview_col = f'{col}_{i}'
            if interview_col in row:
                pc_scores.append(calculate_column_score(row[interview_col], col))
    
    pc_score = np.nanmean(pc_scores) if len(pc_scores) > 0 else np.nan
    
    # Calculate facility score
    facility_score = np.nanmean([hw_score, pc_score])
    
    return pd.Series({
        'Quality_Score': facility_score,
        'HW_Competence_Score': hw_score,
        'HW_Check': hw_scores,
        'Patient_Centeredness_Score': pc_score,
        'PC_Check': pc_scores
        
    })

In [197]:
# Function for integrated supportive supervision

def calculate_ISS(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

    htm = [provides_ART,provides_hiv_test,provides_TB,provides_malaria]
    all = [provides_ART,provides_hiv_test,provides_TB,provides_malaria,provides_anc,provides_imci,provides_immu]
    fac_iss = []
    fac_stats = []
    fac_group_PS = []
    fac_comms_data = []
    iss_scores = []
    
    # Attribute 1 (Supervision in the past 3 months)
    if row['M610'] == 'NO':
        iss_score = 0
    elif row['M610'] == 'YES':
        
        # Attribute 2 (Integrated supervision)
        if sum(htm) >= 3:
            iss_done  = sum(1 if any(row[f'tHFA_C00{i}_{j}'] == 'YES' for i in range(1, 7)) else 0 for j in ['A', 'B', 'C'])
            iss_scores.append(100) if iss_done == 3 else iss_scores.append(0)
        elif sum(all) >= 3:
            iss_done = sum(1 if any(row[f'tHFA_C00{i}_{j}'] == 'YES' for i in range(1, 7)) else 0 for j in ['A', 'B', 'C','D','E','F'])
            iss_scores.append(100) if iss_done >= 3 else iss_scores.append(0)
        else:
            iss_scores.append(np.nan)
        fac_iss.append(iss_scores[-1])
        

        # Attribute 3 (Summary stats discussed)
        iss_scores.append(100) if row['tHFA_C007'] == 'YES' else iss_scores.append(0)
        fac_stats.append(iss_scores[-1])
    
        # Attribute 4 (Group problem solving)
        if row['tHFA_C008'] == 'YES' and row['tHFA_C009'] == 'YES':
            iss_scores.append(100)
        elif any([row['tHFA_C008'], row['tHFA_C009']]) == "DON’T KNOW":
            iss_scores.append(np.nan)
        else: 
            iss_scores.append(0)

        fac_group_PS.append(iss_scores[-1])

        # Attribute 5 (Data on community activities)
        if row['tHFA_C010'] == 'NO':
            iss_scores.append(np.nan)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'DON’T KNOW':
            iss_scores.append(np.nan)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'NO':
            iss_scores.append(0)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'YES':
            iss_scores.append(100)
        else: 
            iss_scores.append(np.nan)
        
        fac_comms_data.append(iss_scores[-1])

        iss_score = np.nanmean(iss_scores)
    else: 
        iss_score = np.nan 
        



    return pd.Series({
        'iss_score': iss_score,
        'iss_check': iss_scores,
        'fac_iss': np.nanmean(fac_iss),
        'fac_stats': np.nanmean(fac_stats),
        'fac_group_PS': np.nanmean(fac_group_PS),
        'fac_comms_data': np.nanmean(fac_comms_data),
        
        })

# see = test.apply(calculate_ISS, axis=1)
# see

In [198]:
def see_services(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_HIV = 1 if provides_hiv_test + provides_ART > 0 else 0

    return pd.Series({
        'HIV': provides_HIV*100,
        'TB': provides_TB*100,
        'Malaria': provides_malaria*100,
        'ANC': provides_anc*100,
        'Immu': provides_immu*100,
    
        })

# services_df = facility_details_df

# services = df.apply(see_services, axis=1)
# services_df = pd.concat([services_df, services], axis=1)
# services_df

# services_df.to_csv('./tHFA-malawi-services_v1.0.csv')

In [199]:
functions = [
    see_services,
    calculate_service_quality,
    calculate_ISS,
    calculate_HTM_ANC,
    calculate_system_CHW,
    calculate_facility_composite,
    calculate_CHW_paid,
    calculate_CLM,
    calculate_guideline_availability,
    calculate_oxygen,
    calculate_testing_capacity,
    calculate_vacancy,    
    ]

In [200]:
df['tHFA_B034'].describe()

count     120
unique     55
top         0
freq        8
Name: tHFA_B034, dtype: int64

In [201]:
# test apply the calculation to each row

test_indicators_df = test_fac_df
for func in functions: 
    see = test.apply(func, axis=1)
    test_indicators_df = pd.concat([test_indicators_df, see], axis=1)
test_indicators_df

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\3396429416.py:88: RuntimeWarning: Mean of empty slice
  'fac_comms_data': np.nanmean(fac_comms_data),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1448578146.py:84: RuntimeWarning: Mean of empty slice
  hiv_testing_score = np.nanmean(hiv_testing_scores) if len(hiv_testing_scores)>0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1448578146.py:85: RuntimeWarning: Mean of empty slice
  ART_access_score = np.nanmean(ART_access_scores) if len(ART_access_scores)>0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1494058245.py:77: RuntimeWarning: Mean of empty slice
  chw_score = np.nanmean(chw_scores)
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1740521874.py:124: RuntimeWarning: Mean of empty slice
  art_retnt_score = np.nanmean(art_retnt_scores)
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\778217302.py:27: RuntimeWarning: Mean of empty slice
  chw_paid_scor

,Q102,Q105,Q106,Q105_a,Q113,Q113_A,Q116,Q116_A,Q117,Q118,...,oxygen_score,oxygen_avl,oxygen_avo,test-capac,test-capac_check,with lab,without lab,vacancy_score,all_funded_posts,all_unfilled_posts
Q100,,,,,,,,,,,,,,,,,,,,,
LBR_000492,God's Will Health Clinic,Montserrado,Central Monrovia district,PRIMARY,HEALTH CENTRE,NaN,PRIVATE-FOR-PROFIT,NaN,URBAN,OUTPATIENT ONLY,...,NaN,NaN,NaN,22.222222,"[100, 0, 0, 0, 100, 100, 0, 100, 0, 0, 0, 0, 0...",22.222222,NaN,0.0,30.0,0.0
LBR_000457,Zobo Medical Clinic,Montserrado,Bushrod District,PRIMARY,CLINIC/DISPENSARY,NaN,PRIVATE-FOR-PROFIT,NaN,RURAL,BOTH OUT AND INPATIENT,...,50.0,0.0,100.0,11.111111,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11.111111,NaN,0.0,12.0,0.0
LBR_000871,St Francis Hospital,Rivercess,Timbo District,TERTIARY,OTHER GENERAL HOSPITAL,NaN,MINISTRY OF HEALTH,NaN,URBAN,BOTH OUT AND INPATIENT,...,50.0,0.0,100.0,25.925926,"[100, 0, 0, 0, 0, 100, 0, 100, 0, 0, 0, 0, 0, ...",25.925926,NaN,50.0,70.0,35.0
LBR_000653,City View Comm. Clinic,Montserrado,Somalia Drive District,PRIMARY,MATERNAL/CHILD HEALTH CLINIC,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,NaN,NaN,NaN,NaN,[nan],NaN,NaN,75.0,8.0,6.0
LBR_000718,Sister Barbara Ann Health Center,Montserrado,Somalia Drive District,PRIMARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,100.0,100.0,100.0,92.592593,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",92.592593,NaN,94.0,50.0,47.0


In [202]:
# FULL Apply the calculation to each row
indicators_df = facility_details_df

for func in functions: 
    see = df.apply(func, axis=1)
    indicators_df = pd.concat([indicators_df, see], axis=1)

indicators_df

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1828855009.py:203: RuntimeWarning: Mean of empty slice
  facility_score = np.nanmean([hw_score, pc_score])
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\3396429416.py:88: RuntimeWarning: Mean of empty slice
  'fac_comms_data': np.nanmean(fac_comms_data),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\3396429416.py:85: RuntimeWarning: Mean of empty slice
  'fac_iss': np.nanmean(fac_iss),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\3396429416.py:86: RuntimeWarning: Mean of empty slice
  'fac_stats': np.nanmean(fac_stats),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\3396429416.py:87: RuntimeWarning: Mean of empty slice
  'fac_group_PS': np.nanmean(fac_group_PS),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\1448578146.py:84: RuntimeWarning: Mean of empty slice
  hiv_testing_score = np.nanmean(hiv_testing_scores) if len(hiv_testing_scores)>0 else np.nan
C:\Users\Seun Osonuga\A

,Q102,Q105,Q106,Q105_a,Q113,Q113_A,Q116,Q116_A,Q117,Q118,...,oxygen_score,oxygen_avl,oxygen_avo,test-capac,test-capac_check,with lab,without lab,vacancy_score,all_funded_posts,all_unfilled_posts
Q100,,,,,,,,,,,,,,,,,,,,,
LBR_000492,God's Will Health Clinic,Montserrado,Central Monrovia district,PRIMARY,HEALTH CENTRE,NaN,PRIVATE-FOR-PROFIT,NaN,URBAN,OUTPATIENT ONLY,...,NaN,NaN,NaN,22.222222,"[100, 0, 0, 0, 100, 100, 0, 100, 0, 0, 0, 0, 0...",22.222222,NaN,0.000000,30.0,0.0
LBR_000457,Zobo Medical Clinic,Montserrado,Bushrod District,PRIMARY,CLINIC/DISPENSARY,NaN,PRIVATE-FOR-PROFIT,NaN,RURAL,BOTH OUT AND INPATIENT,...,50.0,0.0,100.0,11.111111,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11.111111,NaN,0.000000,12.0,0.0
LBR_000871,St Francis Hospital,Rivercess,Timbo District,TERTIARY,OTHER GENERAL HOSPITAL,NaN,MINISTRY OF HEALTH,NaN,URBAN,BOTH OUT AND INPATIENT,...,50.0,0.0,100.0,25.925926,"[100, 0, 0, 0, 0, 100, 0, 100, 0, 0, 0, 0, 0, ...",25.925926,NaN,50.000000,70.0,35.0
LBR_000653,City View Comm. Clinic,Montserrado,Somalia Drive District,PRIMARY,MATERNAL/CHILD HEALTH CLINIC,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,NaN,NaN,NaN,NaN,[nan],NaN,NaN,75.000000,8.0,6.0
LBR_000718,Sister Barbara Ann Health Center,Montserrado,Somalia Drive District,PRIMARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,100.0,100.0,100.0,92.592593,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",92.592593,NaN,94.000000,50.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LBR_000652,Chocolate City Health Center,Montserrado,Somalia Drive District,SECONDARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,NaN,NaN,NaN,NaN,[nan],NaN,NaN,100.000000,40.0,40.0
LBR_000815,Ganta Methodist Hospital,Nimba,Sanniquelleh Mahn District,TERTIARY,OTHER GENERAL HOSPITAL,NaN,MINISTRY OF HEALTH,NaN,PERIURBAN,BOTH OUT AND INPATIENT,...,50.0,0.0,100.0,48.148148,"[100, 0, 0, 0, 100, 100, 0, 100, 0, 100, 0, 10...",48.148148,NaN,NaN,90.0,999.0
LBR_000458,Bensonville Hospital,Montserrado,Careysburg District,TERTIARY,DISTRICT HOSPITAL,NaN,MINISTRY OF HEALTH,NaN,RURAL,BOTH OUT AND INPATIENT,...,100.0,100.0,100.0,22.222222,"[100, 0, 0, 0, 100, 0, 0, 100, 0, 0, 0, 0, 0, ...",22.222222,NaN,NaN,999.0,12.0


In [203]:
# Step 7: Function to aggregate indicators by region and facility type
def aggregate_indicators(df, group_by_columns):
    out  = pd.concat([df.groupby(group_by_columns).size(), df.groupby(group_by_columns).mean()], axis=1)
    out = out.rename(columns={0:'Facility count'})
    return  out

# Example usage:
aggregated_by_type = aggregate_indicators(indicators_df, ['tHFA_A001','Q117'])

In [204]:
aggregated_by_ty = aggregate_indicators(indicators_df, ['tHFA_A001'])
for thing in aggregated_by_ty.index:
    see = pd.DataFrame()
    see = aggregated_by_ty.loc[thing]
    see.name = (f"{thing}","General")
    aggregated_by_type = aggregated_by_type.append(see)
aggregated_by_type

Facility count         HIV          TB     Malaria  \
tHFA_A001 Q117                                                            
PRIMARY   PERIURBAN            18.0   77.777778   27.777778  100.000000   
          RURAL                39.0   92.307692   51.282051  100.000000   
          URBAN                16.0   43.750000   18.750000  100.000000   
SECONDARY PERIURBAN             6.0  100.000000   50.000000  100.000000   
          RURAL                 6.0  100.000000  100.000000  100.000000   
          URBAN                 4.0  100.000000  100.000000  100.000000   
TERTIARY  PERIURBAN            15.0  100.000000   66.666667  100.000000   
          RURAL                 5.0   80.000000  100.000000   80.000000   
          URBAN                11.0   90.909091   72.727273  100.000000   
PRIMARY   General              73.0   78.082192   38.356164  100.000000   
SECONDARY General              16.0  100.000000   81.250000  100.000000   
TERTIARY  General              31.0   93.548387   74.193548   96.774194   

                            ANC        Immu  Quality_Score  \
tHFA_A001 Q117                                               
PRIMARY   PERIURBAN   94.444444   83.333333      82.707751   
          RURAL      100.000000   97.435897      79.495081   
          URBAN       81.250000   75.000000      79.907512   
SECONDARY PERIURBAN  100.000000  100.000000      86.316779   
          RURAL      100.000000  100.000000      90.573533   
          URBAN      100.000000  100.000000      82.555471   
TERTIARY  PERIURBAN  100.000000  100.000000      81.303367   
          RURAL       80.000000  100.000000      82.810648   
          URBAN       81.818182   81.818182      79.166125   
PRIMARY   General     94.520548   89.041096      80.377642   
SECONDARY General    100.000000  100.000000      86.972735   
TERTIARY  General     90.322581   93.548387      80.842167   

                     HW_Competence_Score  Patient_Centeredness_Score  \
tHFA_A001 Q117                                                         
PRIMARY   PERIURBAN            87.045555                   73.157051   
          RURAL                80.304430                   78.999074   
          URBAN                80.988095                   77.535772   
SECONDARY PERIURBAN            93.371212                   73.518519   
          RURAL                93.924844                   87.222222   
          URBAN                93.055556                   72.055386   
TERTIARY  PERIURBAN            85.270034                   77.336700   
          RURAL                86.918519                   80.434028   
          URBAN                83.253561                   76.075213   
PRIMARY   General              82.148255                   77.561961   
SECONDARY General              93.499910                   78.973497   
TERTIARY  General              84.872624                   77.421790   

                      iss_score  ...  guide_imci  oxygen_score  oxygen_avl  \
tHFA_A001 Q117                   ...                                         
PRIMARY   PERIURBAN   60.648148  ...   80.000000     70.000000   40.000000   
          RURAL       91.452991  ...   97.435897     50.000000    0.000000   
          URBAN       54.687500  ...   75.000000     50.000000    0.000000   
SECONDARY PERIURBAN   62.500000  ...  100.000000           NaN         NaN   
          RURAL      100.000000  ...  100.000000     50.000000    0.000000   
          URBAN       91.666667  ...   75.000000     66.666667   33.333333   
TERTIARY  PERIURBAN   82.222222  ...  100.000000     73.333333   53.333333   
          RURAL       70.000000  ...  100.000000     90.000000   80.000000   
          URBAN       84.090909  ...   90.000000     81.818182   63.636364   
PRIMARY   General     75.799087  ...   89.393939     62.500000   25.000000   
SECONDARY General     83.854167  ...   93.333333     58.333333   16.666667   
TERTIARY  General     80.913978  ...   96.666667     79.032258   61.290323   


In [207]:
mean = pd.Series()
mean.at['Facility count'] = len(indicators_df)
sharp = indicators_df.mean()[1:]
mean  = pd.concat([mean,sharp])
mean.name = ('COUNTRY', 'AVERAGE')
mean

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\155398580.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mean = pd.Series()
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_21824\155398580.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sharp = indicators_df.mean()[1:]


Facility count                120.000000
HIV                            85.000000
TB                             53.333333
Malaria                        99.166667
ANC                            94.166667
Immu                           91.666667
Quality_Score                  81.381484
HW_Competence_Score            84.399174
Patient_Centeredness_Score     77.718597
iss_score                      78.194444
fac_iss                        82.882883
fac_stats                      82.300885
fac_group_PS                   80.530973
fac_comms_data                 98.529412
HTM-ANC                        65.196657
hiv testing                    92.057613
ART access                     76.470588
iptp scores                    94.129794
TB screening                   26.106195
system_CHW_score               50.136719
chw_iss                       100.000000
chw_contract                   20.364583
chw_paid                       33.750000
chw_tools                      46.432292
facility_composi

In [208]:
aggregated_by_type = aggregated_by_type.append(mean)
aggregated_by_type = aggregated_by_type.sort_index()
aggregated_by_type

Facility count         HIV          TB     Malaria  \
tHFA_A001 Q117                                                            
COUNTRY   AVERAGE             120.0   85.000000   53.333333   99.166667   
PRIMARY   General              73.0   78.082192   38.356164  100.000000   
          PERIURBAN            18.0   77.777778   27.777778  100.000000   
          RURAL                39.0   92.307692   51.282051  100.000000   
          URBAN                16.0   43.750000   18.750000  100.000000   
SECONDARY General              16.0  100.000000   81.250000  100.000000   
          PERIURBAN             6.0  100.000000   50.000000  100.000000   
          RURAL                 6.0  100.000000  100.000000  100.000000   
          URBAN                 4.0  100.000000  100.000000  100.000000   
TERTIARY  General              31.0   93.548387   74.193548   96.774194   
          PERIURBAN            15.0  100.000000   66.666667  100.000000   
          RURAL                 5.0   80.000000  100.000000   80.000000   
          URBAN                11.0   90.909091   72.727273  100.000000   

                            ANC        Immu  Quality_Score  \
tHFA_A001 Q117                                               
COUNTRY   AVERAGE     94.166667   91.666667      81.381484   
PRIMARY   General     94.520548   89.041096      80.377642   
          PERIURBAN   94.444444   83.333333      82.707751   
          RURAL      100.000000   97.435897      79.495081   
          URBAN       81.250000   75.000000      79.907512   
SECONDARY General    100.000000  100.000000      86.972735   
          PERIURBAN  100.000000  100.000000      86.316779   
          RURAL      100.000000  100.000000      90.573533   
          URBAN      100.000000  100.000000      82.555471   
TERTIARY  General     90.322581   93.548387      80.842167   
          PERIURBAN  100.000000  100.000000      81.303367   
          RURAL       80.000000  100.000000      82.810648   
          URBAN       81.818182   81.818182      79.166125   

                     HW_Competence_Score  Patient_Centeredness_Score  \
tHFA_A001 Q117                                                         
COUNTRY   AVERAGE              84.399174                   77.718597   
PRIMARY   General              82.148255                   77.561961   
          PERIURBAN            87.045555                   73.157051   
          RURAL                80.304430                   78.999074   
          URBAN                80.988095                   77.535772   
SECONDARY General              93.499910                   78.973497   
          PERIURBAN            93.371212                   73.518519   
          RURAL                93.924844                   87.222222   
          URBAN                93.055556                   72.055386   
TERTIARY  General              84.872624                   77.421790   
          PERIURBAN            85.270034                   77.336700   
          RURAL                86.918519                   80.434028   
          URBAN                83.253561                   76.075213   

                      iss_score  ...  guide_imci  oxygen_score  oxygen_avl  \
tHFA_A001 Q117                   ...                                         
COUNTRY   AVERAGE     78.194444  ...   91.891892     73.333333   48.888889   
PRIMARY   General     75.799087  ...   89.393939     62.500000   25.000000   
          PERIURBAN   60.648148  ...   80.000000     70.000000   40.000000   
          RURAL       91.452991  ...   97.435897     50.000000    0.000000   
          URBAN       54.687500  ...   75.000000     50.000000    0.000000   
SECONDARY General     83.854167  ...   93.333333     58.333333   16.666667   
          PERIURBAN   62.500000  ...  100.000000           NaN         NaN   
          RURAL      100.000000  ...  100.000000     50.000000    0.000000   
          URBAN       91.666667  ...   75.000000     66.666667   33.333333   
TERTIARY  General     80.

In [209]:
aggregated_by_type.to_csv(pivot_out)
indicators_df.to_csv(indicators_out)

In [ ]:
# # Step 8: Save outputs to CSV files
# metadata_df = pd.DataFrame(metadata)
# metadata_df.to_csv('/mnt/data/metadata.csv', index=False)
# survey_details_df.to_csv('/mnt/data/survey_details.csv', index=False)
# indicators_df.to_csv('/mnt/data/indicators.csv', index=False)
# aggregated_by_region.to_csv('/mnt/data/aggregated_by_region.csv', index=False)
# aggregated_by_facility_type.to_csv('/mnt/data/aggregated_by_facility_type.csv', index=False)

# print("Data processing completed and files saved.")
